In [1]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import tensorflow.keras as keras

In [3]:
from keras.applications.mobilenet import MobileNet

In [4]:
# Loading the Dataset and getting size of it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

print("Size of the train images:", x_train.shape)
print("Size of the test images:", x_test.shape)

Size of the train images: (50000, 32, 32, 3)
Size of the test images: (10000, 32, 32, 3)


In [5]:
def preprocess_data(X, Y):
        """ This method has the preprocess to train a model """
        X = X.astype('float32')
        X_p = keras.applications.mobilenet.preprocess_input(X)
        Y_p = keras.utils.to_categorical(Y, 10)
        return(X_p, Y_p)


In [6]:
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

In [7]:
if __name__ == "__main__":
    
    base_model = keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False)
    
    image_size = (224, 224)
    
    model = keras.Sequential()
    model.add(keras.layers.Resizing(image_size[0], image_size[1], interpolation="bilinear", input_shape=x_train.shape[1:]))
    model.add(base_model)
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    callback = []
    
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    
    callback += [keras.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [keras.callbacks.ModelCheckpoint('mobilenet_cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=x_train, y=y_train,
               batch_size=128,
               validation_data=(x_test, y_test),
               epochs=10, shuffle=True,
               callbacks=callback,
               verbose=1
               )

2024-03-20 18:41:29.532291: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-20 18:41:29.532315: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-20 18:41:29.532321: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-20 18:41:29.532365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-20 18:41:29.532382: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)



Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/10


2024-03-20 18:41:30.885370: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-20 18:41:32.019257: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 4.0829 - accuracy: 0.3313

2024-03-20 18:46:04.718309: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 287s 721ms/step - loss: 4.0829 - accuracy: 0.3313 - val_loss: 0.8708 - val_accuracy: 0.7488 - lr: 3.2258e-05

Epoch 2: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 2/10
391/391 [==============================] - 280s 717ms/step - loss: 3.0212 - accuracy: 0.6253 - val_loss: 0.8243 - val_accuracy: 0.8626 - lr: 3.2258e-05

Epoch 3: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 3/10
391/391 [==============================] - 476s 1s/step - loss: 2.1345 - accuracy: 0.7736 - val_loss: 0.8488 - val_accuracy: 0.8959 - lr: 3.2258e-05

Epoch 4: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 4/10
391/391 [==============================] - 286s 726ms/step - loss: 1.5543 - accuracy: 0.8428 - val_loss: 0.7882 - val_accuracy: 0.9158 - lr: 3.2258e-05

Epoch 5: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 5/10
391/391 [==================

In [8]:
from keras.models import load_model

model = load_model('mobilenet_cifar10.h5')

In [9]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)

# Calculating metrics
print("Confusion Matrix:")
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), predicted_labels)
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(np.argmax(y_test, axis=1), predicted_labels))


2024-03-20 19:32:51.532924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 15s 46ms/step
Confusion Matrix:
[[947   3  10   5   3   0   1   4  20   7]
 [  2 964   0   0   0   0   1   1   3  29]
 [ 28   0 876  24  31  10  19   8   4   0]
 [  9   2  14 824  16  78  32  10   7   8]
 [  0   0  12  16 922   7  15  27   1   0]
 [  2   0  19  90  17 835  12  24   1   0]
 [  7   1  13  18   4   7 944   4   2   0]
 [  9   0   8   7  18   9   2 946   0   1]
 [ 27   5   4   2   1   0   2   2 948   9]
 [ 12  26   0   2   0   1   1   0   6 952]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1000
           1       0.96      0.96      0.96      1000
           2       0.92      0.88      0.90      1000
           3       0.83      0.82      0.83      1000
           4       0.91      0.92      0.92      1000
           5       0.88      0.83      0.86      1000
           6       0.92      0.94      0.93      1000
           7       0.92      0.95    

In [ ]:
if __name__ == "__main__":
    
    base_model = keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False)
    
    image_size = (224, 224)
    
    model= keras.Sequential()
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(base_model)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation=('relu'))) 
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(256, activation=('relu')))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10, activation=('softmax')))
    
    callback = []
    
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    
    callback += [keras.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [keras.callbacks.ModelCheckpoint('mobilenet_cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=x_train, y=y_train,
               batch_size=128,
               validation_data=(x_test, y_test),
               epochs=10, shuffle=True,
               callbacks=callback,
               verbose=1
               )